In [1]:
!nvidia-smi

Thu Aug 19 04:04:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# 시드 고정
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [4]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [5]:
SOS_TOKEN = 0
EOS_TOKEN = 0

In [6]:
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>":SOS_TOKEN, "<EOS>":EOS_TOKEN}
        self.index2vocab = {SOS_TOKEN:'<SOS>', EOS_TOKEN:'<EOS>'}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(' '):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1

In [7]:
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" "))< source_max_length and len(pair[1].split(" ")) < target_max_length

In [8]:
def preprocess(corpus, source_max_length, target_max_length):
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split('\t')])

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]

    source_vocab = Vocab()
    target_vocab = Vocab()

    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])

    print('source vocab size = ', source_vocab.n_vocab)
    print('target vocab size = ', target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [10]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

In [11]:
def tensorize(vocab, sentence):
    indices = [vocab.vocab2index[word] for word in sentence.split(' ')]
    indices.append(vocab.vocab2index['<EOS>'])

    return torch.Tensor(indices).long().to(device).view(-1, 1)

In [12]:
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [13]:
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)

source vocab size =  17
target vocab size =  13


In [14]:
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

In [15]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=100, learning_rate=0.01):
    loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]

    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    criterion = nn.NLLLoss()

    for i in range(1, n_iter+1):
        source_tensor = training_source[i-1]
        target_tensor = training_target[i-1]

In [16]:
encoder_hidden = torch.zeros([1, 1, enc.hidden_size]).to(device)

In [17]:
output, encoder_hidden = enc(training_source[0][0], enc_hidden_size)

NameError: ignored